In [8]:
company_list = ['Apple', 'Facebook']
years = ['2017', '2016', '2015']

filing_type = '10-k'

In [12]:
# Imports
import pandas as pd
import os
import bs4
import pickle
import json
import requests
from bs4 import BeautifulSoup
import sys
import matplotlib.pyplot as plt


# Defining Functions

# Checking Companies
def company_check(company_list):
    key_path = os.path.join('Ticker_and_CIK', 'Key.csv')
    key_df = pd.read_csv(key_path)
    key_dict = {}

    for query in range(len(company_list)):
        try:
            company_df = key_df[key_df['Company'].str.contains(company_list[query].title())]
            key_dict[company_df.iloc[0, 1]] = [company_df.iloc[0, 0], company_df.iloc[0, 2]]
        except IndexError:
            print(f"{company_list[query]} could not be found")
    if key_dict == {}:
        print ("Whoops, none of your companies were on the S&P 500. Please try again!")
    else:
        print (f"Success! We were able to find {len(key_dict.keys())} of your queries in the S&P 500")
        print (f"We found: {list(key_dict.keys())} in the S&P 500")
        return key_dict

    
# Obtaining Financials from the SEC
def fetch_document_links(cells, year):
    doc_link = 'https://www.sec.gov' + cells[1].a['href']
#     print(doc_link)
    # Obtain HTML for document page
    doc_resp = requests.get(doc_link)
    doc_str = doc_resp.text
    # Find the XBRL link
    soup_doc = BeautifulSoup(doc_str, 'html.parser')
    table_tag_doc = soup_doc.find('table', class_='tableFile', summary='Data Files')
    rows_doc = table_tag_doc.find_all('tr')
    for row_doc in rows_doc:
        cells_doc = row_doc.find_all('td')
        if len(cells_doc) > 3:
                if 'XML' in cells_doc[3].text:
                    return 'https://www.sec.gov' + cells_doc[2].a['href']
                elif 'INS' in cells_doc[3].text:
                    return 'https://www.sec.gov' + cells_doc[2].a['href'] 
                
def fetch_financial_data(ticker, tag_list, as_of_dates, financial_dict):
    row_num = 0
    for tag in tag_list:
        if 'schemaref' in tag.name:
            ignored_href_index = len(ticker) + 1
            as_of_date = tag['xlink:href'][ignored_href_index:(ignored_href_index+8)]
            as_of_dates.append(as_of_date)
            
        if tag.name.find('us-gaap:') != -1:
            try: 
                if float(tag.text) > 0:
                    i = [f'{tag.name}', tag.text, tag['contextref']]
                    financial_dict[f'{row_num}'] = i
                    row_num = row_num + 1
            except ValueError:
                pass

            
# Unmodified Base Code found at https://www.codeproject.com/Articles/1227765/Parsing-XBRL-with-Python
def financial_scraper(key_dict, filing_type, published_date_dict, as_of_date_dict, agg_financial_dict):
    #Looping through Companies in List
    for companies_found in range(len(key_dict.keys())):
        # Obtaining Ticker Symbol
        company_name = list(key_dict.keys())[companies_found]
        ticker = key_dict.get(company_name)[0]

        cik = key_dict.get(company_name)[1]
        dateb = f'{years[0]}-12-31'

        # Obtain HTML for search page
        base_url = "https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type={}&dateb={}"
        edgar_resp = requests.get(base_url.format(cik, filing_type, dateb))
        edgar_str = edgar_resp.text

        doc_links = []
        published_dates = []
        as_of_dates = []

        # Find the document link
        soup = BeautifulSoup(edgar_str, 'html.parser')
        table_tag = soup.find('table', class_='tableFile2')
        rows = table_tag.find_all('tr')
        
        all_years_dict = {}
        
        for year in years:
            for row in rows:
                cells = row.find_all('td')
                if len(cells) > 3:
                    if year in cells[3].text:
                        published_date = cells[3].text
                        published_dates.append(published_date)

                        try:
                            xbrl_link = fetch_document_links(cells, year)
                        except AttributeError:
                            print(f'Error, There is no XBRL file for {ticker} on {published_date}')
                            break
                        # Obtain XBRL text from document
                        xbrl_resp = requests.get(xbrl_link)
                        xbrl_str = xbrl_resp.text

                        # Find and print stockholder's equity
                        soup = BeautifulSoup(xbrl_str, 'lxml')
                        tag_list = soup.find_all()

                        financial_dict = {}

                        fetch_financial_data(ticker, tag_list, as_of_dates, financial_dict)
                 
                        all_years_dict[f'{published_date}'] = financial_dict
                    
        published_date_dict[ticker] = published_dates
        as_of_date_dict[ticker] = as_of_dates
        agg_financial_dict[ticker] = all_years_dict
        
        
        
# Finding Entries already Saved as Variable
def find_entries(as_of_date_dict, published_date_dict, agg_financial_dict, journal_entry_name_dict):
    all_financial_dict = {}
    
    for companies_found in range(len(published_date_dict.keys())):
        ticker = list(published_date_dict.keys())[companies_found]

        comp_curr_ratio_dict = {}
        comp_per_change_dict = {}
        comp_bvps_dict = {}
        comp_eps_dict = {}
        comp_week_change_dict = {}
        comp_month_change_dict = {}
        comp_stock_price_dict = {}
        
        comp_financials_dict = {}

        for date_index in range(len(as_of_date_dict[ticker])):
       
            df = pd.DataFrame.from_dict(agg_financial_dict[ticker][(published_date_dict[ticker][date_index])], orient = 'index')
            date_index_financials_df = {}
            
            for entry in journal_entry_name_dict.keys():
                c_assets_df = df[df[0] == entry]
                example = find_entry(as_of_date_dict, ticker, date_index, c_assets_df, entry)
           
                date_index_financials_df[example[0].values[0]] = int(example[1].values[0]) 
            comp_financials_dict[published_date_dict[ticker][date_index]] = date_index_financials_df
        all_financial_dict[ticker] = comp_financials_dict
    return all_financial_dict
            
def find_entry(as_of_date_dict, ticker, date_index, c_assets_df, entry):
    
    test_date = as_of_date_dict[ticker][date_index]
    y = test_date [0:4]
    m = test_date [4:6]
    d = test_date [6:8]
    
    year_check = int(y) + 1
    
    quarter_list = ['Q4', 'Q3', "Q2", "Q1"]
    context_ref = c_assets_df[2]
    test_df = pd.DataFrame()

    if len(context_ref) == 1:
        # Only one entry
        test_df = test_df.append(c_assets_df)
        print('Only 1 entry')
    elif len(context_ref) == 0:
        print(f'Error, {ticker} has no entries for {entry} on {test_date}')
        
    else:
        while year_check >= (int(y)-1):
            year_check_df = c_assets_df[context_ref.str.contains(f'{year_check}')]
            
            if len(year_check_df) == 1:
                # Only 1 entry for the year
                test_df = test_df.append(year_check_df)
#                 print(test_df)
#                 print('Only 1 entry of that year')
                break
            else: 
                date_formats = [f'{year_check}{m}{d}', f'{year_check}-{m}-{d}', f'{m.lstrip("0")}_{d.lstrip("0")}_{year_check}']

                for item in date_formats:
                    # Check through yyyymmdd, yyyy-mm-dd, and mm_dd_yyyy
                    if len(c_assets_df[context_ref.str.contains(f'{item}')]) == 1:
                        # Only 1 entry
                        test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{item}')])
#                         print(test_df)
#                         print(f'Context Ref uses {item} format')
                        break
                    elif len(c_assets_df[context_ref.str.contains(f'{item}')]) > 1:
                        # Multiple Entries (usually in case of stockholder equity)
                        stock_holder_df = c_assets_df[context_ref.str.contains(f'{item}')]
                        stock_holder_df[1] = stock_holder_df[1].map(int)
                        # Take Max Value given
                        test_df = stock_holder_df[stock_holder_df[1] == stock_holder_df[1].max()].iloc[[0]]
#                         print(test_df)
#                         print(f'Context Ref uses {item} format, multiple entries')
                        break
                else:
                    for quarter in quarter_list:
                        # Check through q#yyyy
                        if len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) > 0:
                            if len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) == 1:
                                test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')])
#                                 print(test_df)
#                                 print(f'Context Ref uses {quarter}{year_check} format')
                                break
                            elif len(c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]) > 1:
                                # Multiple Entries (usually in case of stockholder equity)
                                stock_holder_df = c_assets_df[context_ref.str.contains(f'{quarter}{year_check}')]
                                stock_holder_df[1] = stock_holder_df[1].map(int)
                                test_df = stock_holder_df[stock_holder_df[1] == stock_holder_df[1].max()].iloc[[0]]
#                                 print(test_df)
                                break
                        elif len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) > 0:
                            if len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) == 1:
                                test_df = test_df.append(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')])
#                                 print(test_df)
#                                 print(f'Context Ref uses {year_check}{quarter} format')
                                break
                            elif len(c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]) > 1:
                                # Multiple Entries (usually in case of stockholder equity)
                                stock_holder_df = c_assets_df[context_ref.str.contains(f'{year_check}{quarter}')]
                                stock_holder_df[1] = stock_holder_df[1].map(int)
                                test_df = stock_holder_df[stock_holder_df[1] == stock_holder_df[1].max()].iloc[[0]]
#                                 print(test_df)
                                break
                if len(test_df) == 0:
                    year_check = year_check - 1
                else:
                    year_check = int(y) - 2
#         else:
#             print(f" I'm sorry, {ticker} does not use a proper ContextRef format for {entry} on {test_date} ")
    return test_df

def fetch_stock_price(html_table_dict, quandl_api_key):
    
    all_stock_price_dict = {}

    # Loop through Companies
    for comp in range(len(html_table_dict)):
        comp_stock_price_dict = {}
        ticker = list(html_table_dict.keys())[comp]

        api_link = f'https://www.quandl.com/api/v3/datasets/WIKI/{ticker}.json?api_key={quandl_api_key}'
        response = requests.get(api_link)
        json_data = json.loads(response.text)

        for date in list(html_table_dict[ticker].keys()):
            date_stock_price_list = []

            for day in range(len(json_data['dataset']['data'])):
                if date in json_data['dataset']['data'][day]:
                    date_stock_price_list.extend((json_data['dataset']['data'][day-1][11],\
                                                  json_data['dataset']['data'][day+1][11],\
                                                  json_data['dataset']['data'][day+5][11]))
            
            comp_stock_price_dict[date] = date_stock_price_list
        all_stock_price_dict[ticker] = comp_stock_price_dict
    
    return all_stock_price_dict

def rearrange_table(html_table_dict, journal_entry_name_dict):
    rearranged_table_dict = {}
    for ticker in html_table_dict.keys():
        
        rearranged_entry_name_dict = {}
        for entry_name in journal_entry_name_dict.keys():
            
            rearranged_entry_list = []
            for date in html_table_dict[ticker].keys():
                if entry_name in html_table_dict[ticker][date]:
                    rearranged_entry_list.append(html_table_dict[ticker][date][entry_name])
                else:
                    rearranged_entry_list.append(None)
            rearranged_entry_name_dict[entry_name] = rearranged_entry_list
            rearranged_table_dict[ticker] = rearranged_entry_name_dict
    
    return rearranged_table_dict

def one_day_stock_movement(all_stock_price_dict):
    one_day_stock_movement_dict = {}
    for ticker in all_stock_price_dict.keys():
        
        date_list = []
        day_move_list = []
        
        for date in all_stock_price_dict[ticker].keys():
            price_movement = (all_stock_price_dict[ticker][date][2] / all_stock_price_dict[ticker][date][0])
            day_move_list.append(round(((price_movement-1)* 100), 2))
            date_list.append(date)
        one_day_stock_movement_dict[ticker] = [date_list, day_move_list]
    
    return one_day_stock_movement_dict

def find_current_ratio(all_financials_dict):
    all_current_ratio_dict = {}
    for ticker in all_financials_dict.keys():

        date_list = []
        current_ratio_list = []

        for date in all_financials_dict[ticker].keys():
            for entry_name in all_financials_dict[ticker][date].keys():

                if entry_name == 'us-gaap:assetscurrent':
                    current_assets = all_financials_dict[ticker][date][entry_name]

                elif entry_name == 'us-gaap:liabilitiescurrent':
                    current_liabilities = all_financials_dict[ticker][date][entry_name]
                    current_ratio = round(current_assets / current_liabilities, 2)
                    current_ratio_list.append(current_ratio)
                    
            date_list.append(date)
        all_current_ratio_dict[ticker] = [date_list, current_ratio_list]

    return all_current_ratio_dict

    
# Defining Date Dictionaries
published_date_dict = {}
as_of_date_dict = {}
agg_financial_dict = {}

journal_entry_name_dict = {'us-gaap:cashandcashequivalentsatcarryingvalue': 'Cash', 'us-gaap:assetscurrent': 'Current Assets', 'us-gaap:assets': 'Assets',\
  'us-gaap:liabilitiescurrent': "Current Liabilities", 'us-gaap:liabilities': 'Liabilities', 'us-gaap:stockholdersequity': 'Stockholders Equity'}

quandl_api_key = 'C1LNyzrxv1pvjkP955ua'

# Running Functions
key_dict = company_check(company_list)
financial_scraper(key_dict, filing_type, published_date_dict, as_of_date_dict, agg_financial_dict)   
all_financials_dict = find_entries(as_of_date_dict, published_date_dict, agg_financial_dict, journal_entry_name_dict)
# print("HTML")
# print(all_financials_dict)
rearranged_table_dict = rearrange_table(all_financials_dict, journal_entry_name_dict)
# print("Rearrange")
# print(rearranged_table_dict)
all_stock_price_dict = fetch_stock_price(all_financials_dict, quandl_api_key)
print("All_Stock")
print(all_stock_price_dict)
one_day_stock_movement_dict = one_day_stock_movement(all_stock_price_dict)
print("One_Day")
print(one_day_stock_movement_dict)  
current_ratio = find_current_ratio(all_financials_dict)
# print(current_ratio)

Success! We were able to find 2 of your queries in the S&P 500
We found: ['Apple Inc.', 'Facebook, Inc.'] in the S&P 500
Error, There is no XBRL file for FB on 2016-04-27
Error, There is no XBRL file for FB on 2015-02-13


C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Kanishka Ramanan\anaconda3\lib\site-packages\ipykernel_launcher.py:238: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documenta

All_Stock
{'AAPL': {'2017-11-03': [174.25, 168.11, 163.05], '2016-10-26': [112.48224004304, 116.18645077821, 115.07617010693], '2015-10-28': [115.95126768775, 110.19843784645, 109.43844861992]}, 'FB': {'2017-02-03': [132.06, 130.84, 132.18]}}
One_Day
{'AAPL': [['2017-11-03', '2016-10-26', '2015-10-28'], [-6.43, 2.31, -5.62]], 'FB': [['2017-02-03'], [0.09]]}
